# Subset Sum Problem

In [1]:
import tensorflow as tf

In [2]:
tf.executing_eagerly()

True

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Naive Implementation (Approximate)

Let the superset be the following vector and our target be $7.0$.

$$
target = 7.0
$$

\begin{equation*}
superset = 
\begin{bmatrix}
1.0 & 2.0 & 3.0 & 4.0 & 5.0 \\
\end{bmatrix}
\end{equation*}

Our goal is to find a mask, such that, the dot product results in the target. Here is an example of a mask that adds up to our target.

\begin{equation*}
mask = 
\begin{bmatrix}
0.0 & 0.0 & 1.0 & 1.0 & 0.0 \\
\end{bmatrix}
\end{equation*}

We can verify that $$ mask \cdot superset = target $$


### Bistable loss
See [boolean-satisfiability.ipynb](boolean-satisfiability.ipynb) for more details

In [4]:
@tf.function
def bistable_loss_fn(x):
    a = (x ** 2)
    b = (x - 1) ** 2
    
    return a * b

### Total Loss

To force the optimzer to pick boolean like values over minimizing squared difference, we give more weight to the bistable loss.

$$ loss_{total} = \sqrt{(mask \cdot superset - target) ^ 2} + e^{loss_{bistable}} $$

In [5]:
@tf.function
def total_loss(target, subset_sum, mask):
    l2_loss = tf.math.squared_difference(target, subset_sum)
    bistable_loss = tf.reduce_sum(bistable_loss_fn(mask))
    
    return l2_loss + tf.exp(bistable_loss)

In [6]:
@tf.function
def subset_sum_fn(mask, container):
    return tf.tensordot(mask, container, axes=1)

In [7]:
container = tf.Variable([1,2,3,4,5],dtype=tf.float32)
mask = tf.Variable(tf.ones(tf.shape(container)),dtype=tf.float32)
target = tf.constant(7.0, dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    subset_sum = subset_sum_fn(mask, container)
    loss = total_loss(target, subset_sum, mask)

print(loss)
print(mask.numpy())
print(tape.gradient(loss,mask))
print(tape.gradient(loss,container))

tf.Tensor(65.0, shape=(), dtype=float32)
[1. 1. 1. 1. 1.]
tf.Tensor([16. 32. 48. 64. 80.], shape=(5,), dtype=float32)
tf.Tensor([16. 16. 16. 16. 16.], shape=(5,), dtype=float32)


In [8]:
container = tf.Variable([1,2,3,4,5],dtype=tf.float32)
mask = tf.Variable(tf.ones(tf.shape(container)),dtype=tf.float32)
target = tf.constant(7.0, dtype=tf.float32)

@tf.function
def train_step():
    with tf.GradientTape() as tape:
        subset_sum = subset_sum_fn(mask, container)
        loss = total_loss(target, subset_sum, mask)
    grads = tape.gradient(loss, mask)
    opt.apply_gradients(zip([grads], [mask]))
    return loss

In [9]:
# opt = tf.keras.optimizers.Adam(learning_rate=3e-4)
opt = tf.keras.optimizers.Adam()
for i in range(10000):
    loss = train_step()
    if i % 1000 == 0:
        answer = tf.reduce_sum(tf.round(mask) * container)
        tf.print(i, loss, mask, answer)

0 65 [0.998999953 0.998999953 0.998999953 0.998999953 0.998999953] 15
1000 1.45896971 [0.49006924 0.487600476 0.486836314 0.486464143 0.486245453] 0
2000 1.36283946 [0.462699383 0.465229839 0.466141194 0.466604263 0.466885746] 0
3000 1.36171114 [0.44351235 0.461678118 0.466522902 0.468744069 0.470017612] 0
4000 1.35450292 [0.378572643 0.455236733 0.46950981 0.475195825 0.478215873] 0
5000 1.30244493 [0.147182733 0.444465905 0.484647065 0.496831417 0.502350032] 5
6000 1.27151334 [0.00469869794 0.363226235 0.50079155 0.525202513 0.532478869] 12
7000 1.20019722 [-0.00187286059 0.058930587 0.531536877 0.587706268 0.588588715] 12
8000 1.18279672 [-0.00845454726 -0.0166277438 0.448080063 0.665599108 0.60900861] 9
9000 1.05360293 [-0.0136801191 -0.0250828899 -0.0223536715 0.903152943 0.706790388] 9


### Result

We get the mask

\begin{equation*}
mask = 
\begin{bmatrix}
0.0 & 0.0 & 0.0 & 1.0 & 1.0 \\
\end{bmatrix}
\end{equation*}

Which gives the sum of $9.0$ instead of $7.0$.

The system seems to be stuck in a local optima. Training further would not improve the results. 

**Note: Since, our batch size is one. We are training with Gradient Descent instead of Stochastic Gradient Descent. Global Optima is not guaranteed**